In [ ]:
using LinearAlgebra

# N spins
N = 13
Dim = 2^N
# couplings
J_X = 1
J_Y = 0


J_Z = 1
B = 0
#decay
alpha = 10

#function for the distance 1/(r_ij ^alpha)
function r_distance(index_i, index_j, alpha)
    dist = abs(index_i - index_j)^alpha
    return (1 / dist)
end

cutoff = 1e-7
# initialise Hamiltonian
Hamiltonian = zeros(Float32, Dim, Dim)

for Ket = 0:Dim-1
    Diagonal = 0.0

    for i = 0:N-1 #we're doing i!=j so here looping first for i<j (effectively half a sum). Then we multiply by 2 every Hamiltonian contribution to account for all pairs. 
        for j = i+1:N-1

            var_ij = r_distance(i, j, alpha)

            #if cutoff wanted, uncomment the following lines

            #if var_ij < cutoff
            #    contnue
            #end

            #S^Z part, similar to before 
            Spin1 = 2 * ((Ket >> i) & 1) - 1
            Spin2 = 2 * ((Ket >> j) & 1) - 1
            Diagonal += -2.0 * r_distance(i, j, alpha) * J_Z * 0.25 * Spin1 * Spin2


            #S^X part 
            bit_i = 2^i
            bit_j = 2^j
            Bra = Ket ⊻ bit_i ⊻ bit_j
            Hamiltonian[Bra+1, Ket+1] += -2.0 * J_X * 0.25 * r_distance(i, j, alpha)

            si = (Ket >> i) & 1
            sj = (Ket >> j) & 1

            #S^Y part- not all terms have the same sign from the fact that S^Y=1/2i (S^+-S^-). I expanded S^Y*S^Y to check which terms would have an overall minus sign. 

            Bra = Ket ⊻ (bit_i) ⊻ (bit_j)
            sign = (si == sj) ? 1 : -1
            Hamiltonian[Bra+1, Ket+1] += -(-0.25) * 2 * J_Y * r_distance(i, j, alpha) * sign


        end

    end

    Hamiltonian[Ket+1, Ket+1] = Diagonal
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1, Ket+1] = -0.5 * B
    end

end




In [2]:
using LinearAlgebra
Diag = eigen(Hamiltonian)
GroundState = Diag.vectors[:, 1]  #this gives the groundstate eigenvector
energy = Diag.values
println(length(GroundState))
println("Groundstate eigenvector:$(GroundState)")
println("Corresponding eigenvalue: $(energy[1]/N)")

8192
Groundstate eigenvector:Float32[-0.4208209, 0.011175491, 0.0055219894, -0.20754646, 0.008453977, -0.052081607, -0.10538522, 0.01711718, 0.006148101, -0.037825946, -0.03238131, 0.0052575488, -0.12892841, 0.003432178, 0.0040131556, -0.15076542, 0.008013729, -0.019161155, -0.016395742, 0.0068497784, -0.03454756, 0.0019715324, 0.0023039675, -0.040379487, -0.112442926, 0.0064085955, 0.0031680255, -0.055510238, 0.011015698, -0.026350297, -0.05332055, 0.022327494, 0.006319592, -0.0150905745, -0.01016204, 0.0042487197, -0.021405064, 0.0012221863, 0.001815034, -0.031801213, -0.031863715, 0.0018165621, 0.0010169147, -0.017814457, 0.0035339175, -0.008454262, -0.015107081, 0.0063237464, -0.11889742, 0.0031614867, 0.001769182, -0.066419534, 0.002710693, -0.016679408, -0.02980153, 0.0048470767, 0.0048433132, -0.029770404, -0.0200487, 0.0032587168, -0.079862766, 0.0021277084, 0.0031613442, -0.11878172, 0.007917254, -0.0098441, -0.0066265576, 0.005322308, -0.011728187, 0.0013781539, 0.002046869, 

In [9]:
magnetizationX = 0.0
for Ket = 0:Dim-1  #Loop over Hilbert Space
    for Bra = 0:Dim-1  #The "easy" (slow) way to do this 
        SumSx = 0.0
        for SpinIndex = 0:N-1  #Loop over spin index 
            bit = 2^SpinIndex   #The bit to be flipped by S+ or S-
            Bra2 = Ket ⊻ bit    #Binary XOR flips the bit
            if Bra == Bra2
                SumSx += 0.5 #the factor of 1/2 above
            end
        end
        magnetizationX += SumSx * GroundState[Ket+1] * GroundState[Bra+1]
    end #Bra
end #Ket
println(magnetizationX / N)

0.5000000968575504
